In [ ]:
import numpy as np
import control as con
from numpy import linalg as LA

import cvxpy
import optim_tools as optim_tools#own file with helper

In [ ]:
###########################
# Hydraulischer Aktor     #
###########################

A0 = np.matrix([[0,   1,       0],
                [-10, -1.167, 25],
                [0,   0,    -0.8]])
print "Eigenvalues: {}".format(LA.eigvals(A0))
#a = -A[-1,:].T ### !!!!!
#print a
b0 = np.matrix([[0],[0],[2.4]])
c0 = np.matrix([1, 0, 0])
d0 = np.matrix([0])
u_max = 10.5
n = 3

X0 = [np.matrix([-20.0, -10, -10]).T,
      np.matrix([-20.0, -10, 10]).T,
      np.matrix([-20.0,  10, -10]).T,
      np.matrix([20.0,  -10, 10]).T,
      np.matrix([20.0,  -10, -10]).T,
      np.matrix([20.0,   10, 10]).T]

#print "A:\n", A
#print "a:\n", a
#print "b:\n", b
#print "c:\n", c


# Convert to Normalform
ss, T = con.canonical_form(con.ss(A0, b0, c0, d0), form='reachable')

A = optim_tools.reverse_x_order(np.matrix(ss.A))
a = -A[-1][:].T #!!!!

b = optim_tools.reverse_x_order(np.matrix(ss.B))
c = optim_tools.reverse_x_order(np.matrix(ss.C))
d = optim_tools.reverse_x_order(np.matrix(ss.D)) # == 0!

print "A:\n", A
print "a:\n", a
print "b:\n", b
print "c:\n", c

In [ ]:
######################################################
# Helper for Constraint variant (4.65) -> Q_sum      #
######################################################

# TODO: Was ist m? Kann man das berechnen oder wird das festgelegt m<=2n-1, meistens n+1?

from scipy.special import comb as nchoosek # n Choose k (n ueber k)

def get_a_list(a, Q, z, m):
    n = len(a) # dim of a, z, Q(nxn)
    m = m # Is this to choose or somehow given?
    l = m+1 # length of the array with all a_i in it i:{0,m}
    H = lambda k: optim_tools._H(k, n) # Convinence function to fix n and make specialized H(k)
    N = optim_tools._N(n)

    a_list = [np.zeros((n, n))] * l
    for i in range(0, l): # m eingeschlossen
        if i <= (m-1)/2.0: # 0 <= i <= (m-1)/2
            for k in range(1, i+1):
                a_list[i] += a.T * H(n+k-i)*Q*N*H(n-k+1)*a - z.T*N*H(n-i)*a
        elif i <= m: # (m-1)/2 < i <=m
            for k in range(1, 2*n-i+1):
                a_list[i] += a.T * H(k)*Q*N*H(2*n-i-k+1)*a
        else:
            # this branch is currently not possible in this program
            print "i={} < m={}".format(i, m)
            a_list[i] = 0
            
    return a_list


def trans_a_list(a_list, eps):
    l = len(a_list)
    m = l-1 #biggest index in a_list
    a1_list = [np.zeros(a_list[0].shape)] * l
    for j in range(0, l): #for each in a_list
        for i in range(j, l): # for each coefficient including m
            a1_list[j] += nchoosek(i, i-j) * ((1.0+eps)/(1.0-eps))**(i-j) * ((1.0-eps)/(2.0))**i * a_list[i]
    return a1_list

def calc_a_Sum(a_list):
    l = len(a_list) # number of matrizen in a_list
    m = l-1 # Index of Q_m
    n = a_list[0].size[0] # shape of each matrix, first element
    
    if m is 0:
        a_sum = cvxpy.bmat([[2*a_list[0],   np.zeros(n)], 
                            [np.zeros(n), np.zeros(n)]])
    elif m is 1:
        a_sum = cvxpy.bmat([[2*a_list[0], a_list[1]],
                            [a_list[1],   np.zeros(n)]])
    else: # e.g. m is 2 or more
        a_sum = cvxpy.bmat([[2*a_list[0], a_list[1]],
                            [a_list[1], 2*a_list[2]]])

    for i1 in range(3, l, 2):
        S_new_col = cvxpy.vstack(np.zeros((((i1+1)/2-1)*n, n)), a_list[i1])

        if i1 is m:
            S_new_row = cvxpy.hstack(np.zeros((n, ((i1+1)/2-1)*n)), a_list[i1], np.zeros((n,n)))
        else:
            S_new_row = cvxpy.hstack(np.zeros((n, ((i1+1)/2-1)*n)), a_list[i1], 2*a_list[i1+1])

        a_sum = cvxpy.bmat([[a_sum, S_new_col],
                            [S_new_row]])

    a_sum = -0.5*a_sum
    
    return a_sum

def calc_lmi_cond(a_sum, n):
    k = a_sum.size[1] / n # This dimension is from Boris. Dilyana does not specifiy the dimension though

    J = np.hstack([np.zeros((n*(k-1), n)), np.eye(n*(k-1))])
    C = np.hstack([np.eye(n*(k-1)), np.zeros((n*(k-1), n))])
    return np.vstack([C, J]), n*(k-1)

In [ ]:
##############################
# Convex Problem (4.58)      #
##############################

# Variables
Q  = cvxpy.Variable(n,n) # Implys (459) (semidefinite for numerical reasons?)

z = cvxpy.Variable(n)

# Constants
N = optim_tools._N(n)

# Constraints
constraint_459 = Q >> 0
constraint_460 = Q*(A.T + a*b.T) + (A + b*a.T)*Q - z*b.T - b*z.T << 0
constraint_461 = Q*N+N*Q << 0

constraint_462 = [cvxpy.bmat([[1,       X0[i].T],
                              [X0[i],    Q      ]]) >> 0
                                for i in range(0, len(X0))]

constraint_463 = cvxpy.bmat([[u_max**2 - a.T*Q*a + 2*a.T*z, z.T],
                             [z,                           Q  ]]) >> 0


# Collection of all constraints
constraints = [constraint_460] # -> for some objectives this constraint is replaced, therefore constraints[0]==constraint_460
constraints.append(constraint_459)
constraints.append(constraint_461)
constraints.extend(constraint_462) ##!! Beware of the "extend" if input is array
constraints.append(constraint_463)


In [ ]:
###########################################
# Constraint variant (4.65) -> Q_sum      #
###########################################

m = n

# Preparation for constraint (4.65)
a_list = get_a_list(a, Q, z, m) # Matrizes of Polynom coefficients: S(p)=S_0 + S_1*p + ... S_n*p^n
a1_list = trans_a_list(a_list, 0.01) # Intervaltransformation p:[0,1] -> p1:[-1,1] (S1 -> S^tilde)
a1_sum = calc_a_Sum(a1_list) # S^tilde_sum Matrix (30)
CJ, l = calc_lmi_cond(a1_sum ,n) # "Selection matrizes" of (31), l=dimension of P and G

# Further variables of optimization
S = cvxpy.Variable(l,l) #symmetrical
G = cvxpy.Variable(l,l) #skew

# Constraints on new variables
constraint_G = G + G.T == 0 # skew symmetry
constraint_S = S == S.T # symmetry

# Actual constraint
constraint_465 = a1_sum << CJ.T * cvxpy.bmat([[-S,  G],
                                              [G.T, S]]) * CJ

# Appending all constraints to "problem"

constraints.append(constraint_G)
constraints.append(constraint_S)
constraints.append(constraint_465)

In [ ]:
print "TO FUTURE-ME!"
print "STOP! Please rethink and choose the following cells wisely!"
print "AND NOT ALL OF THEM!"

In [ ]:
###########################################
# Objective variant (4.66) -> max(det(Q)) #
# Volumenmaximierung -> geomean(Q)        #
###########################################

# Help variable to minimize geomean
#v = cvxpy.Variable(1)

# Objective representation
#constraint_obj_451 = v >= -cvxpy.geo_mean(Q)

#constraints.append(constraint_obj_451)

obj = cvxpy.Minimize(-cvxpy.geo_mean(Q)) # Not yet implemented

prob = cvxpy.Problem(obj, constraints)

# Solve the problem with SCS
prob.solve(solver=cvxpy.SCS, verbose=False, max_iters=2500)
print prob.status
Q.value

In [ ]:
########################
# Objective playground #
########################

#obj = cvxpy.Minimize(0) # Feasiblity
obj = cvxpy.Maximize(cvxpy.log_det(Q))
#obj = cvxpy.Maximize(cvxpy.lambda_min(Q))

prob = cvxpy.Problem(obj, constraints)

# Solve the problem with SCS
prob.solve(solver=cvxpy.SCS, verbose=False, max_iters=2500)
print prob.status
print "Q:\n", Q.value
print "R1:\n", LA.inv(Q.value)

In [ ]:
###########################################
# Objective variant (4.70)                #
# Max. Abklingrate                        #
###########################################

beta = cvxpy.Parameter(sign='positive') # Bisection parameter

constraint_obj_471 = Q*(A.T + a*b.T) + (A + b*a.T)*Q - z*b.T - b*z.T + 2*beta*Q << 0

constraints[0] = constraint_obj_471 # Replace constraint_460
obj = cvxpy.Minimize(0) # Feasiblity

prob = cvxpy.Problem(obj, constraints)

# Solve with bisection (and SCS)
[[Q_o, z_o], beta_o] = optim_tools.bisect_max(0, 10, prob, beta, [Q, z], solver=cvxpy.SCS)

print "Q:\n", Q_o
print "z:\n", z_o
print "beta:", beta_o

In [ ]:
###########################################
# Objective variant (Kap. 4.5.3)          #
# Min. quad. Gütemaß                      #
###########################################

gamma = cvxpy.Parameter(sign='positive') # Bisection parameter

# Further variables of optimization
W = cvxpy.Parameter(n, 1) # ~c -> Min(Integral{y.T*y}) -> Min(Integral{x.T*c.T*c*x}) -> Min(Integral{x.T*Ĝ*x})
W.value = np.matrix([1,0,0]).T # G=[1 0 0; 0 0 0; 0 0 0] ~~ c.T*c

# Constraints on new variables
#constraint_W = W*W.T >> 0 # symmetry ?
#print constraint_W

constraint_obj_xxx = cvxpy.bmat([[Q*(A.T + a*b.T) + (A + b*a.T)*Q - z*b.T - b*z.T, Q*W   ],
                                 [W.T*Q,                                           -gamma]]) << 0

constraints[0] = constraint_obj_xxx # Replace constraint_460
                                
obj = cvxpy.Minimize(0) # Feasiblity

prob = cvxpy.Problem(obj, constraints)


#gamma.value = 30.8990478516
#prob.solve(solver=cvxpy.SCS, verbose=True)

# Solve with bisection (and SCS)
[[Q_o, z_o], gamma_o] = optim_tools.bisect_min(20, 35, prob, gamma, [Q, z], solver=cvxpy.SCS, verbose=False)

print "Q:\n", Q_o
print "R1:\n", LA.inv(Q_o)

print "z:\n", z_o
print "gamma:", gamma_o

Some checks for validy of solution

In [ ]:
import numpy as np
print "Checking constraints (no output is ok!)"
eig_459 = np.linalg.eigvals(Q_o)
if np.any(eig_459 <= 0): print("Q not pos definit: {}".format(eig_459))

#eig_460 = np.linalg.eigvals((Q*(A.T+a*b.T)+(A+b*a.T)*Q-z*b.T-b*z.T).value)
#if np.any(eig_460 >= 0): print("4.60 not neg definit: {}".format(eig_460))

#eig_461 = np.linalg.eigvals((Q*N+N*Q).value)
#if np.any(eig_461 >= 0): print("4.61 not neg definit: {}".format(eig_461))

